# INSIGHT FRM NETFLIX ENGAGEMENT

## INSERT LIBRARY

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime as dt
import plotly.express as px
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib.font_manager")

pd.options.display.max_columns

20

##  GATHERING DATA

This stage begins with collecting the data needed for analysis purposes and creating it in df variables (dataframe)

In [2]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vassyesboy/netflix-engagement-jan-jun-23
!mkdir netflix
!unzip /content/netflix-engagement-jan-jun-23.zip -d netflix

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
df = pd.read_csv ('/content/netflix/Netflix Engagement (plus).csv')
df.head()


In [ ]:
df.shape

## ASSESING DATA

Next, test the quality of the data for further analysis to get maximum results

In [ ]:
df.info()

From the data information, there is no visible difference in the data, it's just that the release date section will be changed to datetime, and in the other columns we will leave it like that for now, and in the data section there are several columns that contain quite a lot of missing values, it will completed in the data cleaning section

In [ ]:
df.isna().sum()

From the data you can see that a lot of data contains missing values

In [ ]:
round(((df.isnull().sum()) / df.shape[0])*100, 2)

make missing values in percents

In [ ]:
print(f'number of duplicated : {df.duplicated().sum()}')


## CLEANING DATA

#### Data Duplicated

In [ ]:
df.drop_duplicates(inplace = True)
df.info()
df.shape

Handle duplicate data

#### Available Globally?

In [ ]:
#change the column name to global version to make it easier
df.rename(columns = {'Available Globally?':'Global Version'}, inplace=True)

change columns name Available Globally? be Global Version

#### Release Date

In [ ]:
#change the release column to date time

df['Release Date']= pd.to_datetime(df['Release Date'])

df.info()

change the object data type in the release date column to datetime

#### Rating

In [ ]:
#see rating distribution
sns.histplot(df['Rating'], bins =20 , kde=True)
plt.axvline(x=np.mean(df.Rating), color='r', label='Mean', linestyle='dashed')
plt.title('Distribution of Rating')
plt.legend()
plt.show()

Because the insight that we will be looking for later touches the realm of ratings, for the initial stage, first check the distribution of film ratings. In this section, the distribution of film ratings is not a normal distribution because the data distribution is more skewed to the left, therefore it is the right way to deal with missing values. This is by using the median instead of using the average.


In [ ]:
#Handling missing Rating values ​​with median
df['Rating']=df['Rating'].fillna(df['Rating'].median())
df.info()
df.shape

#### Genre

Next, we will deal with the missing values ​​in the Genre section, because the data type in the genre column is an object and is in the form of a string, so overcoming the missing values ​​is by using simpleImputer from skLearn to fill in predictions about which data often appears.

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
df['Genre'] = imputer.fit_transform(df[['Genre']]).ravel()
df.info()
df.shape

#### Release Date

In [ ]:
#resolve missing values ​​in the Release Date column
df['Release Date'].fillna('Unknown', inplace =True)
df.info()
df.shape

#### Number Of Rating

At this stage, we will check how the number of rating data is distributed, to overcome missing values ​​in that column with histogram plot.

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df["Number of Ratings"], bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Number of Ratings')
plt.xlabel('Number of Ratings')
plt.ylabel('Frequency')
plt.show()

Next we will look again at data distribution using shapiro from skLearn

In [ ]:
from scipy.stats import shapiro
stat, p = shapiro(df["Number of Ratings"])
print("Shapiro-Wilk test statistic:", stat)
print("p-value:", p)
if p > 0.05:
    print("Distribusi Number of Ratings normal")
else:
    print("Distribusi Number of Ratings tidak normal")

Because the data is normally distributed, it is better to deal with missing values ​​in the data using the mean.

In [ ]:
df['Number of Ratings']=df['Number of Ratings'].fillna(df['Number of Ratings'].mean())
df.info()
df.shape

#### Key Word

just like in the genre column which is of the object data type, in this section we will also use simpleImputer from skLearn.

In [ ]:
#mengatasi missing value di keyword dengan imputer sklear
df['Key Words'] = imputer.fit_transform(df[['Key Words']]).ravel()
df.info()
df.shape

#### Description

In [ ]:
df['Description']=imputer.fit_transform(df[['Description']]).ravel()
df.info()
df.shape

#### Checking Outliers

In this section we will check whether the data contains outliers for later data analysis purposes, starting by creating a new variable containing data from a numerical data frame.

In [ ]:
numericCols = df.select_dtypes(include = ['Int64','Float64'])

In [ ]:
df.info()
df.shape

In [ ]:
def OutliersIqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR  # Sesuaikan multiplier di sini
    upper_bound = Q3 + 3 * IQR
    outliers = (data < lower_bound) | (data > upper_bound)
    return outliers

# Determine the number of subplots and layout
num_subplots = len(numericCols.columns)
num_rows = 1
num_cols = num_subplots

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5))

# Combining subplots
for i, col in enumerate(numericCols.columns):
    outliers = OutliersIqr(numericCols[col])

    # Determines the position of the subplot
    if num_subplots > 1:
        ax = axes[i]
    else:
        ax = axes

    # Plotting Data Point (blue)
    sns.scatterplot(x=numericCols.index, y=numericCols[col], color='blue', label=f'{col} Data Point', alpha=0.5, ax=ax)

    # Plotting Outliers (red)
    sns.scatterplot(x=numericCols[outliers].index, y=numericCols[outliers][col], color='red', label=f'{col} Outliers', ax=ax)

    ax.set_title(f"Outliers Detection for {col}")

# Arrange the layout
plt.tight_layout()
plt.show()

df.shape


From the distribution seen, there are many outliers in the data distribution and in the next stage we will overcome these outliers by removing outliers.From the distribution seen, there are many outliers in the data distribution and in the next stage we will overcome these outliers by removing outliers.

In [ ]:
def handleOutliers(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = (data < lower_bound) | (data > upper_bound)

    # Winsorizing outliers
    data[outliers] = data.quantile([0.05, 0.95])

    return data

# Menentukan jumlah subplot dan layout
num_subplots = len(numericCols.columns)
num_rows = 1
num_cols = num_subplots

# Membuat subplot
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5))

# Menggabungkan subplot untuk penanganan outliers
for i, col in enumerate(numericCols.columns):
    # Menentukan posisi subplot
    if num_subplots > 1:
        ax = axes[i]
    else:
        ax = axes

    # Menangani outliers
    numericCols[col] = handleOutliers(numericCols[col])

    # Plotting Data Point setelah penanganan (blue)
    sns.scatterplot(x=numericCols.index, y=numericCols[col], color='blue', label=f'{col} Data Point', alpha=0.5, ax=ax)

    ax.set_title(f"Outliers Handling for {col}")

# Menyusun layout
plt.tight_layout()
plt.show()
df.shape


In [ ]:
 df.isna().sum()

### Summary:

At this data preprocessing stage, in data cleaning we have done:

1. Clean duplicate data in the dataset
2. Change the data type on the release date which was originally an object type to date time to facilitate later analysis.
2. Deal with missing values ​​in the rating column where the distribution is not normal or in this case the data distribution is skewed to the left, therefore we deal with missing values ​​using the median.
3. Next, clean the columns of the object or string data type using simpleImputer from skLearn, which fills in missing values ​​using data that often appears based on predictions.
4. Handling outliers.

## EDA

At this stage we will find out what insights can be seen from the data based on the question:
1. What genres of films get the attention of the audience, both in terms of ratings and the number of hours watched?
2. What film got the most ratings and the most hours watched?
3. How is film distribution distributed globally?

Note: You don't have to answer this question in sequence

In [ ]:
df.describe().T


In [ ]:
df.describe(include='O').T

### Checking correlation

Start EDA by looking at how the numeric columns are correlated

In [ ]:
numericCol = df.select_dtypes(include=['float64', 'int64']).columns
corrColumns = df[numericCol].corr()

plt.figure(figsize=(10, 5))
sns.heatmap(corrColumns, cmap='viridis', annot=True, fmt='.2f', linewidth=0.5)
plt.show()
df.shape


### Global Version


In [ ]:
custom_colors = ['#48dbfb', '#9980FA']

data_counts = df['Global Version'].value_counts()
data_percentage = data_counts / df.shape[0]

plt.pie(data_percentage, autopct='%1.1f%%', labels=data_counts.index, colors=custom_colors, explode=[0.05, 0.05], shadow=True, startangle=90)

plt.title('Global Version Distribution')
plt.show()


In this section of the chart we see how films are distributed globally, and the result is that around 75.2% of the total films released are not distributed globally, only 24.8% percent of films or series are distributed globally.

### Distribution of film releases during 2023

In [ ]:
df['Release Date'] = pd.to_datetime(df['Release Date'], errors='coerce')  # Coerce errors to NaT for invalid values
df = df.dropna(subset=['Release Date'])  # Drop rows with NaT values

movies2023 = df[df['Release Date'].dt.year == 2023]

release_counts = movies2023['Release Date'].value_counts().sort_index()

rolling_avg = release_counts.rolling(window=7).mean()

plt.figure(figsize=(10, 6))
plt.plot(release_counts.index, release_counts.values, marker='o', linestyle='-', label='Daily Releases')
plt.plot(rolling_avg.index, rolling_avg.values, marker='', linestyle='-', color='red', label='7-day Rolling Average')

plt.title('Number of Movie Releases in 2023')
plt.xlabel('Release Date')
plt.ylabel('Number of Movies Released')
plt.legend()

plt.grid(True)
plt.show()


From the chart results above we can get insight:

**Netflix Movie Release Patterns**

The graph shows that the number of Netflix film releases tends to be higher in the early months of the year (January, February, and March) and then decreases in the following months. This may be caused by several factors, such as:

* Netflix releases more new movies at the start of the year to attract new subscribers and retain existing ones.
* Netflix spent more on film production at the start of the year.
* Netflix has more opportunities to promote its new films earlier in the year, such as at awards shows and film festivals.

**Surge in Film Releases**

There are some spikes in movie releases visible in the graph, which may indicate special events or promotions being run by Netflix on these dates. For example, a surge in movie releases on March 23, 2023 might be caused by the premiere of the Netflix series "The Night Agent."

### Distribution of ratings for films released during 2023

In [ ]:
fig = px.scatter(movies2023, x='Release Date', y='Rating', color_continuous_scale='oranges',
                 title='Ratings of Films Released in 2023', hover_data=['Title'], color='Title',
                 labels={'Rating': 'Rating', 'Release Date': 'Release Date'}, width=900)

fig.update_layout(
    showlegend=False
)

fig.show()


From this chart it can be seen that the distribution of ratings for films or series released in 2023 received an average rating of good numbers between 5 and above, in other words, the average rating for films released in 2023 received a good rating.

### Genre Insights Based on the most dominant, most watched, and ratings

At this stage we will see insights into which genre dominates based on the number of views and ratings.
start by creating a new data frame into newDf so that it is easy to manipulate.
then check each value in the value column, because the initial value column contains a list of genres, we will check and break down the column values ​​to become a single genre value, not in list form using apply and explode.

In [ ]:
newDf=df.copy()

newDf['Genre']=newDf['Genre'].apply(lambda x: eval(x) if pd.notna(x) else [])
newDf

In [ ]:
genre=newDf.explode('Genre')
genre.head()
genre

After selecting the values ​​from the columns, we can calculate which genres appear frequently by adding up the total genres and sorting them.

In [ ]:
# Menggunakan DataFrame hasil explode
genreGroup = genre.groupby('Genre').agg({
    'Rating': 'mean',
    'Hours Viewed': ['sum', 'mean'],  # Menambahkan rata-rata jam tonton
}).reset_index()

# Menyusun ulang kolom setelah penggunaan multi-level index
genreGroup.columns = ['Genre', 'Rating', 'Total Hours Viewed', 'Average Hours Viewed']

def format_hours_viewed(value):
    if value >= 1_000_000_000:
        return f'{value / 1_000_000_000:.2f}B'
    elif value >= 1_000_000:
        return f'{value / 1_000_000:.2f}M'
    elif value >= 1_000:
        return f'{value / 1_000:.2f}K'
    else:
        return f'{value}'

# Menerapkan format pada kolom 'Total Hours Viewed'
genreGroup['Total Hours Viewed'] = genreGroup['Total Hours Viewed'].apply(format_hours_viewed)
genreGroup['Average Hours Viewed'] = genreGroup['Average Hours Viewed'].apply(format_hours_viewed)

result = genreGroup[['Genre', 'Average Hours Viewed']].sort_values(by='Average Hours Viewed', ascending=False)
print(result)
# Menampilkan hasil
print(result)


In [ ]:
genreCount = genre['Genre'].value_counts(ascending=False)
genreCount

In [ ]:
df.shape

From this data it can be seen that the comedy genre occupies the first position with the genre that appears frequently.

In [ ]:
topGenre = genreCount.head(5)

plt.figure(figsize= (10,5))
sns.barplot( y=topGenre.values, x=topGenre.index, palette='pastel')
plt.title('Highest Number of Genre Category Frequencies')
plt.ylabel('Genre Frequency Count')
plt.xlabel('Genre Types')
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
description = (
    f"Most common genre: {topGenre.idxmax()} with {topGenre.max()} movies.\n"
    f"Least common genre: {genreCount.idxmin()} with {topGenre.min()} movies."
)

def pieChart(data, title, description):
    fig = px.pie(data, values=data.values, names=data.index, title=title)
    fig.update_layout(title_text=title, title_x=0.5)
    fig.update_layout(
        annotations=[
            dict(
                x=0.5,
                y=-0.15,
                xref='paper',
                yref='paper',
                text=description,
                showarrow=False,
                font=dict(size=10),
            )
        ]
    )
    fig.show()

pieChart(topGenre, 'Top 5 Genre Distribution', description)


Summary:

From the EDA results in the Genre section we get the insight that:
1. The comedy genre is the most dominant genre with 2055 films.
2. The genre with the lowest position is the Western genre with a total of 17 films.
3. The top 5 genres based on number of films are Comedy, Drama, Short, Documentary and Action.



In [ ]:
newDf.info()

In [ ]:
genre['Formatted Hours Viewed'] = genre['Hours Viewed'].apply(formatViews)

genreByRating = genre.groupby('Genre')['Rating'].mean()
genreByHoursViewed = genre.groupby('Genre')['Hours Viewed'].sum()

genreRatingandViewed = pd.DataFrame({'Avg Rating': genreByRating, 'Total Views': genreByHoursViewed.apply(formatViews)}).reset_index()

genreSort = genreRatingandViewed.sort_values(by=['Avg Rating', 'Total Views'], ascending=[False, True]).reset_index()

genreSort




In [ ]:

# Retrieve the top 10 genres
top10GenreSort = genreSort.sort_values('Total Views', ascending=False).head(10)

# Create two side-by-side plots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Change the number of rows to 2 to create vertical plots

# Plot for average rating (light blue)
ax1.bar(top10GenreSort['Genre'], top10GenreSort['Avg Rating'].round(1), color='skyblue')
ax1.set_xlabel('Genre', fontsize=12)
ax1.set_ylabel('Average Rating', fontsize=12)
ax1.set_title('Top 10 Genres by Average Rating', fontsize=14)
ax1.bar_label(ax1.containers[0])

# Plot for total views (orange)
ax2.bar(top10GenreSort['Genre'], top10GenreSort['Total Views'], color='orange')
ax2.set_xlabel('Genre', fontsize=12)
ax2.set_ylabel('Total Views', fontsize=12)
ax2.set_title('Top 10 Genres by Total Views', fontsize=14)

# Adjust the spacing between plots
plt.tight_layout()

# Display the plot
plt.show()


### Movies based on hours watched and rating

In [ ]:
top10Movies = df.nlargest(10, 'Hours Viewed')
top10Movies[['Title', 'Genre', 'Rating', 'Hours Viewed']]

worst10Movies = df.nsmallest(10, 'Hours Viewed')
worst10Movies[['Title', 'Genre', 'Rating', 'Hours Viewed']]

figTop = px.bar (top10Movies, x ='Title', y='Hours Viewed', color = 'Genre',
              hover_data = 'Rating', title = ' Best 10 Movies by Hours Viewed in Half 2023')
figTop.show()

In this section we see that the 10 films or series with the highest number of hours watched are The Night Agent: Season 1 and this is followed by other films or series.

In [ ]:
figWorst = px.bar(worst10Movies, x='Hours Viewed', y='Title', color='Genre',
                    hover_data='Rating', title='Worst 10 Movies by Hours Viewed in Half 2023')

# Mengakses objek trace pertama
trace = figWorst.data[0]

# Mengubah warna bar pertama
trace.marker.color = 'red'

figWorst.show()


Next, this section shows the 10 films with the fewest views.

In [ ]:
# Retrieve the top 10 movies with the highest ratings and the bottom 10 movies with the lowest ratings
top_10_rating = df.nlargest(10, 'Rating')
bottom_10_rating = df.nsmallest(10, 'Rating')

# Plot the top 10 movies with the highest ratings
fig_top = px.bar(top_10_rating, x='Title', y='Rating', color='Title', title='Top 10 Movies by Rating')
fig_top.update_xaxes(tickangle=45, tickmode='array')

# Plot the bottom 10 movies with the lowest ratings
fig_bottom = px.bar(bottom_10_rating, x='Title', y='Rating', color='Title', title='Bottom 10 Movies by Rating')
fig_bottom.update_xaxes(tickangle=45, tickmode='array')

# Add the rating values above each bar in both subplots
fig_top.update_traces(texttemplate='%{y:.2f}', textposition='outside')
fig_bottom.update_traces(texttemplate='%{y:.2f}', textposition='outside')

# Change the title from "Movie Title" to "Title" and use specified font style
font_style = {'family': 'sans-serif', 'size': 14, 'color': 'black'}
fig_top.update_layout(title='Top 10 Movies by Rating', title_font_size=14, title_font_color='black', title_font_family='sans-serif')
fig_bottom.update_layout(title='Bottom 10 Movies by Rating', title_font_size=14, title_font_color='black', title_font_family='sans-serif')

# Display the plots
fig_top.show()
fig_bottom.show()


Summary

In this section we have carried out an analysis with insight:

1. Get an idea of ​​what genre data dominates with the most appearances in all genres in films.
2. Get insight into which films occupy the top positions based on ratings and the number of times they have been watched.